In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)

imputedDataset = pd.read_stata("/Users/burke/Documents/research/bpCog/mcm/mcm/data/fullyImputedDataset.dta")
imputedDataset.head()
noRecentUSInmigration = imputedDataset.loc[~imputedDataset.timeInUS.isin([1,2,3])]
earlyCrossSectionOfNonImmigrants = noRecentUSInmigration.loc[noRecentUSInmigration.year==2001]
# here we just need to throw out the very young people for comparisons
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.loc[earlyCrossSectionOfNonImmigrants.age > 18 + 2]
earlyCrossSectionOfNonImmigrants = earlyCrossSectionOfNonImmigrants.sample(n=len(earlyCrossSectionOfNonImmigrants), weights=earlyCrossSectionOfNonImmigrants.WTINT2YR, replace=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# on UCOD 1 = MI, 2=stroke, 3 = other cause of death
earlyCrossSectionOfNonImmigrants.ucod.value_counts(normalize=True, sort=False)

0    0.874492
1    0.042242
2    0.006296
3    0.076970
Name: ucod, dtype: float64

In [19]:
import os
os.chdir("/Users/burke/Documents/research/bpCog/mcm")
from mcm.population import NHANESDirectSamplePopulation
from mcm.outcome import OutcomeType

pop = NHANESDirectSamplePopulation(n=10000, year=2001)
pop.advance(10)

In [20]:
popData = pd.DataFrame({"index" : pop._people.index, 
                    "baseAge" : [person._age[0] for person in pop._people],
                    "finalAge" : [person._age[-1] for person in pop._people],
                   'mi' : [person.has_mi_during_simulation() for person in pop._people], 
                   'stroke' : [person.has_stroke_during_simulation() for person in pop._people], 
                   'miAge' : [person._outcomes[OutcomeType.MI][0][0] if len(person._outcomes[OutcomeType.MI]) > 0 else -1 for person in pop._people],
                   'strokeAge' : [person._outcomes[OutcomeType.STROKE][0][0] if len(person._outcomes[OutcomeType.STROKE]) > 0 else -1 for person in pop._people],
                   'fatalStroke' : [person.has_fatal_stroke() for person in pop._people],
                   'fatalMI' : [person.has_fatal_mi() for person in pop._people],

                   'dead' : [person.is_dead()  for person in pop._people]})

popData['age10'] = pd.qcut(popData.baseAge, 10, labels=False)
popData['nonCVDeath'] = False
popData.loc[((popData.dead) & (~popData.fatalStroke) & (~popData.fatalMI)), 'nonCVDeath'] = True

popData['cvDeath'] = False
popData.loc[((popData.dead) & (popData.fatalStroke) | (popData.fatalMI)), 'cvDeath'] = True



In [23]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath', 'cvDeath']].groupby('age10').mean()

,baseAge,stroke,mi,dead,fatalStroke,fatalMI,nonCVDeath,cvDeath
age10,,,,,,,,
0,20.989843,0.006464,0.042475,0.010157,0.000000,0.003693,0.006464,0.003693
1,25.993743,0.026069,0.064651,0.025026,0.004171,0.010428,0.010428,0.014599
2,30.838933,0.012846,0.065217,0.024704,0.002964,0.011858,0.009881,0.014822
3,36.047000,0.033000,0.112000,0.049000,0.005000,0.024000,0.020000,0.029000
4,40.820111,0.054428,0.144834,0.050738,0.011993,0.015683,0.023063,0.027675
5,45.492205,0.065702,0.180401,0.076837,0.012249,0.041203,0.023385,0.053452
6,50.502732,0.117486,0.249545,0.115665,0.022769,0.037341,0.055556,0.060109
7,56.587918,0.185545,0.307443,0.162891,0.036677,0.055016,0.071197,0.091694
8,65.677518,0.319430,0.432350,0.314344,0.057986,0.086470,0.169888,0.144456


In [22]:
popData[['baseAge', 'stroke', 'mi', 'dead', 'age10', 'fatalStroke', 'fatalMI', 'nonCVDeath']].mean()

baseAge        44.7937
stroke          0.1339
mi              0.2085
dead            0.1426
age10           4.4354
fatalStroke     0.0273
fatalMI         0.0375
nonCVDeath      0.0778
dtype: float64